# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.dropna()
cities_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hobart,100,AU,1579965707,77,-42.88,147.33,66.00,1.01
1,1,Constitucion,0,CL,1579965902,38,-35.33,-72.42,72.21,8.25
2,2,Carnarvon,0,AU,1579965902,74,-24.87,113.63,77.43,22.84
3,3,Aklavik,100,CA,1579965718,95,68.22,-135.01,-13.76,24.36
4,4,Nelson Bay,99,AU,1579965923,90,-32.72,152.15,77.00,3.89
...,...,...,...,...,...,...,...,...,...,...
530,530,Saiha,0,IN,1579966908,63,22.48,92.97,48.74,2.68
531,531,Ballia,0,IN,1579966908,75,25.76,84.15,55.13,4.47
532,532,Tutoia,0,BR,1579966908,55,-2.76,-42.27,90.46,7.02
533,533,Chancay,98,PE,1579966968,87,-11.57,-77.27,72.25,8.23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity'].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Conditions = Cloudiness < 50, Humidity < 50, Max Temp < 90, Wind Speed < 5
criteria_df = cities_df.drop(cities_df.index[(cities_df['Cloudiness'] > 50)])
criteria_df = criteria_df.drop(criteria_df.index[(criteria_df['Humidity'] > 50)])
criteria_df = criteria_df.drop(criteria_df.index[(criteria_df['Max Temp'] > 90)])
criteria_df = criteria_df.drop(criteria_df.index[(criteria_df['Wind Speed'] > 5)]).dropna().reset_index(drop=True)
criteria_df = criteria_df.rename(columns={'Unnamed: 0':'Cities Index','Date':'Datetime'})
criteria_df

,Cities Index,City,Cloudiness,Country,Datetime,Humidity,Lat,Lng,Max Temp,Wind Speed
0,77,Simdega,0,IN,1579966129,31,22.62,84.52,60.69,2.84
1,213,Swidnica,0,PL,1579966406,10,50.84,16.49,33.01,2.64
2,327,Baloda Bazar,0,IN,1579966606,37,21.67,82.17,69.66,1.52
3,409,Botou,0,CN,1579966740,36,38.07,116.57,33.96,3.94
4,419,Corbita,0,RO,1579966742,48,46.15,27.30,41.00,1.99
5,442,Bendigo,0,AU,1579966746,47,-36.77,144.28,64.00,3.69
6,472,Ponta Pora,20,BR,1579966835,48,-22.54,-55.73,84.20,4.70
7,508,Tumut,0,AU,1579966903,39,-35.30,148.22,62.98,3.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
# Create hotel_df, only need city, country, lat/lng for google api, and hotel name
# Based on the criteria_df, so we already set the parameters we want for the city to vacation at
hotel_df = criteria_df.drop(['Cities Index','Cloudiness','Datetime','Humidity','Max Temp','Wind Speed'],axis=1)
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Simdega,IN,22.62,84.52,
1,Swidnica,PL,50.84,16.49,
2,Baloda Bazar,IN,21.67,82.17,
3,Botou,CN,38.07,116.57,
4,Corbita,RO,46.15,27.30,
5,Bendigo,AU,-36.77,144.28,
6,Ponta Pora,BR,-22.54,-55.73,
7,Tumut,AU,-35.30,148.22,


In [30]:
# Google places search for hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": target_coordinates,
    "type": "hotel",
    "radius": 5000,
    "key": g_key
}
# Use iterrows to go through the dataframe
for index, row in hotel_df.iterrows():
    # Get the lat, lng from each row
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    latlng = f"{lat},{lng}"
    params['location'] = latlng
    
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    # Run through the cities
    try:
        print(f'Finding you a hotel in {city}.')
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print('Missing field/result, skipping city.')
    print('------------')

Finding you a hotel in Simdega.
------------
Finding you a hotel in Swidnica.
------------
Finding you a hotel in Baloda Bazar.
------------
Finding you a hotel in Botou.
------------
Finding you a hotel in Corbita.
------------
Finding you a hotel in Bendigo.
------------
Finding you a hotel in Ponta Pora.
------------
Finding you a hotel in Tumut.
------------


In [31]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Simdega,IN,22.62,84.52,Simdega
1,Swidnica,PL,50.84,16.49,Świdnica
2,Baloda Bazar,IN,21.67,82.17,Baloda Bazar
3,Botou,CN,38.07,116.57,Cangzhou
4,Corbita,RO,46.15,27.30,Homocea
5,Bendigo,AU,-36.77,144.28,Bendigo
6,Ponta Pora,BR,-22.54,-55.73,Pedro Juan Caballero
7,Tumut,AU,-35.30,148.22,Tumut


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
        locations,
        info_box_content=hotel_info,
        display_info_box=True,
        hover_text=hotel_df['City']
    )
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))